# 8.1. Introduction to Word Embeddings

In [ ]:
import gensim.downloader
import numpy as np

In [ ]:
# This will download a set of pre-trained word embeddings.
# There are many vectors trained on a variety of corpora with different vector dimensions.
# See the documentation of the gensim package.
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

NameError: name 'gensim' is not defined

## Additional Resources

* [Documentation](https://radimrehurek.com/gensim/models/keyedvectors.html) on the pretrained GLoVe vector models.
* [Website](https://nlp.stanford.edu/projects/glove/) for the original GLoVe paper and its pretrained models.
* [Book Chapter](https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb) on Recurrent Neural Networks, which are sequence models that can be used to build classifiers using word embeddings.
* [Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) article on embeddings.

## Motivation

So far in this course, we've done analysis using a *bag of words* model: we assume that every document can be analyzed as a collection of word counts, ignoring their order, and treating each unique token in the document as distinct in meaning. But that's not great for many reasons, including:

1. Word *inflections* ("dog" vs. "dogs") are lost, and we treat the inflected and uninfected words as completely distinct. If we want to take inflection into account, we *lemmatize* the words to their base form and throw away this meaning.
2. Words which are synonyms ("great" vs. "excellent"), or have related meanings ("king" vs. "queen") similarly are treated as unrelated.
3. A single word can have multiple meanings: the verb "dust" both means to remove dust (dust the shelves) to apply dust (dust for fingerprints). These are counted as the same word.
3. Unrelated homonyms like "turn" (to rotate) and "turn" (your time to do something) are counted as the same word.
4. Word order *does* matter, and it matters quite a lot.

An embedding is a way to mitigate these issues, allowing us to relate words to each other and to allow words to have multiple meanings. These won't fix our word order problems, but they are an important step towards doing so with *recurrent* and *convolutional neural networks* (RNNs and CNNs), and understanding *vector embeddings*, which are very broadly useful in machine learning.

#### What is an Embedding?

Generally speaking, an *embedding* is a procedure that converts data from a *sparse* format to a *dense* format.

**Sparse** data are mostly 0. Word counts are extremely sparse. If we consider a particular document, and list its word counts, assuming a vocabulary size of $N$ (let's say $N = 10,000$), we'll need $N$ numbers to represent the word. A typical word vector will look like this, which is sparse:

$$''\text{aardvark}''\rightarrow[1, 0, 0, 0, \cdots]$$
$$''\text{potato}''\rightarrow[0, 0, \cdots 0, 1, 0, \cdots]$$

You can think of sparse data as very inefficiently using the storage given to them. Sparse data are like a huge bookshelf with only one or two books on it. What's more, models using sparse data are highly prone to overfitting because they require very complex models: for a vocabulary size of $N$, you need *at least* $N$ parameters.

**Dense** data are not full of zeros. Every element of a dense vector is nonzero, for example:

$$''\text{aardvark}''\rightarrow[2.1, -0.4, 7.2 \cdots]$$

If we can find a high-quality embedding, then words can be described using $D$ numbers, often between 50 and 300 (and $D \ll N$; often $D \approx \sqrt{N}$ works well). This also means that we can make a mode less prone to overfitting, as instead of *at least* $N$ parameters, we can start with only $D$ parameters and achieve the same level of success.

#### A Second Approach

What we need is a better way to represent a word's meaning than as a collection of letters. The letters themselves don't convey meaning (and often don't tell you how to pronounce a word, either!). So instead, we'll use numbers.

A single number won't work. Why? We might imagine making the number represent how positive or negative a word is. Then "awesome" becomes 1, "good" is 0.5, "bad" is -0.5, "okay" is 0, and so on. But there are plenty of other shades of meaning in words beyond how positive or negative they are. Some carry different kinds of feelings. Others represent technical ideas, objects, animals. Anything really.

Instead, we'll use a vector, where each word will be represented by multiple numbers. Imagine that each entry in the vector represents some shade of meaning in the word. One represents positivity/negativity, another gender. Whatever. The more entries (the *dimension*) in the vector, the more subtleties we can include for each word. Typical dimensions are 50 to 300. Here's the word "man" in one pre-trained set of vectors:

In [ ]:
glove_vectors.get_vector("man")

array([-0.094386,  0.43007 , -0.17224 , -0.45529 ,  1.6447  ,  0.40335 ,
       -0.37263 ,  0.25071 , -0.10588 ,  0.10778 , -0.10848 ,  0.15181 ,
       -0.65396 ,  0.55054 ,  0.59591 , -0.46278 ,  0.11847 ,  0.64448 ,
       -0.70948 ,  0.23947 , -0.82905 ,  1.272   ,  0.033021,  0.2935  ,
        0.3911  , -2.8094  , -0.70745 ,  0.4106  ,  0.3894  , -0.2913  ,
        2.6124  , -0.34576 , -0.16832 ,  0.25154 ,  0.31216 ,  0.31639 ,
        0.12539 , -0.012646,  0.22297 , -0.56585 , -0.086264,  0.62549 ,
       -0.0576  ,  0.29375 ,  0.66005 , -0.53115 , -0.48233 , -0.97925 ,
        0.53135 , -0.11725 ], dtype=float32)

It is not immediately clear where a "gender" or any other dimension may be, and it might be a combination of several of these.

As is often the case in machine learning, we do not tell the algorithm what dimensions we want it to find. It merely finds the ones that fit best. The [the GLoVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf), looks at what words occur in the vicinity of each other. It tries to make the *dot product* of any two word vectors equal to the frequency of their counts. So if a word vector for the $i$th word is written as:

$$\vec{w_i} = \left<w_1, w_2, w_3, \cdots w_D\right>$$

Then we want:

$$\vec{w_i}\cdot\vec{w_j} + b_i + b_j = w_{i1} w_{j1} + w_{i2} w_{j2} + \cdots w_{iD}w_{jD} + b_i + b_j = \ln X_{ij}$$

Where $X_{ij}$ is the frequency of words $i$ and $j$ appearing in the same document and $b_i$ and $b_j$ are bias terms (which correspond, roughly, to telling us when a word is very frequent). The bias terms are thrown away after fitting is complete.

### Uses of Word Vectors

##### Distance Between Words

If two words are similar in meaning, then they should have similar values in each of the abstract dimensions that make up the word vectors. They should have similar levels of "positivity", "gender", and so on. This means that they point, roughly, in the same direction.

We can ask for *gensim* to calculate the cosine of the angle between two vectors, which gives us a rating from 0 (unrelated) to 1 (very similar). Similarities less than 0 are possible as well, which means the words actively avoid each other in texts.

We use the cosine because if the angle between two vectors is small, then $\cos\theta \approx \cos 0^\circ = 1$. If they point in very different directions ($90^\circ$ apart), then $\cos\theta \approx \cos90^\circ = 0$. Due to the weirdness of high dimensional spaces, it is quite rare to get vectors that point in opposite directions ($\cos\theta < 0$)

In [ ]:
glove_vectors.similarity("newyork", "binghamton")

0.46534905

Slightly more quantitatively, we know that if two words show up together quite frequently, then $X_{ij}$ should be quite high. Thus, $\vec{w}_i\cdot \vec{w}_{j}$ should also be relatively large. That means that the cosine of the angle between them:

$$\cos\theta = \frac{\vec{w}_i\cdot \vec{w}_{j}}{||\vec{w}||\vec{w_j}||}$$

Should also be large.

##### Similar Words

We can also ask the *gensim* library to tell us what words are most similar to others. It simply calculates the distance to each other word in its vocabulary, and returns the largest ones.

In [ ]:
glove_vectors.most_similar('man')

[('woman', 0.8860337734222412),
 ('boy', 0.8564431071281433),
 ('another', 0.8452839851379395),
 ('old', 0.8372183442115784),
 ('one', 0.827606201171875),
 ('who', 0.8244695663452148),
 ('him', 0.8194693922996521),
 ('turned', 0.8154467940330505),
 ('whose', 0.811974048614502),
 ('himself', 0.807725727558136)]

##### Analogies

As shown on the [GLoVe website](https://nlp.stanford.edu/projects/glove/), the GLoVe algorithm produces linear connections between related concepts. For example, the image below shows the direction and distance between male-female word pairs:

<div style="text-align:center;"><img src="https://nlp.stanford.edu/projects/glove/images/man_woman.jpg" width="500px"></div>

We can use this to ask our vectors to complete analogies for us, i.e.:

<div style="text-align:center">man : king :: woman : ?</div>

Or, what word has the same relationship with "woman" that "king" has with "man"? The answer should be "queen". This can be done by adding and subtracting vectors. We imagine subtracting "man" from "king" then adding "woman" to what remains.

In [ ]:
glove_vectors.most_similar(positive=['best', "strongest", 'dark'], negative=['good', "strong"])

[('featured', 0.6686994433403015),
 ('animated', 0.6653846502304077),
 ('superhero', 0.664042055606842),
 ('shadows', 0.6478701829910278),
 ('films', 0.6459299325942993),
 ('villains', 0.6420229077339172),
 ('featuring', 0.6413138508796692),
 ('depicted', 0.636328935623169),
 ('black-and-white', 0.6362449526786804),
 ('feature', 0.6313946843147278)]

##### Machine Learning

Instead of using word counts, we can use the word vectors and the $x$ variable when we do machine learning. This can be done by averaging the word vectors in a document together, optionally removing stop words.